In [5]:
## all imports
from IPython.display import HTML
import chardet
import io
import numpy as np
np.seterr(divide='ignore', invalid='ignore')
import os
import urllib.request
from urllib.request import urlopen
from bs4 import BeautifulSoup
from time import sleep
import requests
import operator
import socket
import _pickle
import math
import re # regular expressions

from pandas import Series
import pandas as pd
from pandas import DataFrame

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (20, 10)
import json

import seaborn as sns
sns.set_context("talk")
sns.set_style("white")

from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time
from datetime import datetime

#from secret import *

In [4]:
print ("this")

this


### FUNCTION TO CREATE URL

In [6]:
def find_url(row):
    Base_Link_01= "https://en.wikipedia.org/wiki/"
    Base_Link_02= "_AFL_season"
    Variable= row
    link = Base_Link_01 + Variable + Base_Link_02
    return (link)

### FUNCTION TO ENSURE ALL MONTH SYNTAX IS CONSISTENT

In [7]:
def correct_month (x):
    first_space_index = x.index(" ")
    day = x[:first_space_index]
    #print (day)
    time = x[first_space_index:].strip()
    space_index = time.index(" ")
    hours = time[space_index:].strip()
    split_month = x.split(" ")[1]
    #print (split_month)
    
    months_tocheck = ["January", "Febuary ", "March", "April", "May", "June", "July", "August", "September", "October", "Novemeber", "December"]
    
    #if any (split_month for month in months_tocheck):
    if split_month in months_tocheck:
        #print ("it is a correct month")
        return (x)
    
       
    else:
        month_dict = {
        "Mar" : "March",
        "Apr" : "April",
        "May" : "May",
        "June" : "June",
        "July" : "July",
        "Aug" : "August",
        "Sep" : "September",
        "Oct" : "October",
        }
        #print (month_dict[split_month])
        #print ("is not a correct minth")
        return (day + " " + month_dict[split_month] + " " + hours)


        


### FUNCTION TO RETURN DATE

In [8]:
def the_date (x):
    first_space_index = x.index(" ")
    day = x[:first_space_index]
    return (day)

### FUNCTION TO RETURN MONTH

In [9]:
def the_month (x):
    first_space_index = x.index(" ")
    day = x[:first_space_index]
    #print (day)
    time = x[first_space_index:].strip()
    space_index = time.index(" ")
    hours = time[space_index:].strip()
    split_month = x.split(" ")[1]
    return (split_month)
    #print (split_month)

### FUNCTION TO RETURN TIME (HOURS)

In [10]:
def the_time (x):
    first_space_index = x.index(" ")
    day = x[:first_space_index]
    #print (day)
    time = x[first_space_index:].strip()
    space_index = time.index(" ")
    hours = time[space_index:].strip()
    split_month = x.split(" ")[1]
    return (hours[1:-1])
    #print (split_month)

### FUNCTION TO RETURN DAY IF IT EXISTS

In [11]:
def if_dayexists (x):
    ## we know if it has a day it will have a comma
    #x = "Thursday, 14 June (5:40 pm)"
    count = x.count(",")
    if count >0:
        date = x.split(",")[0].strip()
        #print (date)
        #first_colon_index = x.index(",")
        #print (first_colon_index)
        #date = x[first_colon_index+1:].strip()
        return (date)
    else: 
        date = x
        return (x, "No Day Data Provided")
    


### FUNCTION TO CREATE STRIPTIME + DATETIME

In [12]:
def structured_times (x):

    struct_time = time.strptime(x, "%Y %d %B (%I:%M %p)")
    dt = datetime.fromtimestamp(time.mktime(struct_time))
    return (dt)

### FUNCTION TO CONVERT TO CONSISTENT DATE SYNTAX (NO PF1 AT FRONT)

In [13]:
def date_conversion (x):
    #x = "PF1: Friday, 25 September (6:20 pm)"
    #x = "Thursday, 14 June (5:40 pm)"
    count = x.count(":")
    if count >1:
        first_colon_index = x.index(":")
        date = x[first_colon_index+1:].strip()
        return (date)
    else: 
        date = x
        return (date)

### REMOVE DAY FOR STRPTIME CONSISTENCY

In [14]:
def day_conversion (x):
    ## we know if it has a day it will have a comma
    #x = "Thursday, 14 June (5:40 pm)"
    count = x.count(",")
    if count >0:
        date = x.split(",")[1].strip()
        #print (date)
        #first_colon_index = x.index(",")
        #print (first_colon_index)
        #date = x[first_colon_index+1:].strip()
        return (date)
    else: 
        date = x
        return (date)
    
    


### MATCH DICTIONARY

In [15]:
def match_dictionary_return (day, date, month, this_year, this_round, time, struct_time, home_team, home_goals,home_behinds, home_total, away_team, away_goals, away_behinds, away_total, venue, final_crowd, match_report_url):
    ind_match_dict= {
         "Day": day,
         "Date": date,
         "Month": month,
         "Year": this_year,
         "Round": this_round,         
         "Time": time,
         "Structured Data": struct_time,
         "Home Team": home_team,
         "Home Goals": home_goals,
         "Home Behinds": home_behinds,
         "Home Total": home_total,
         "Away Team": away_team,
         "Away Goals": away_goals,
         "Away Behinds": away_behinds,
         "Away Total": away_total,
         "Venue": venue,
         "Crowd": final_crowd,
         "Match Link": match_report_url}

    return (ind_match_dict)

### FIND THE MATCH DETAILS WHEN IT IS THIS, MAYBE RETURN THIS AS A DICTIONARY SO WE CAN JUST CALL THEM OUT.... B
### BUT JUST MAKE IT WORK FIRST SO IT GETS IT OUT

In [ ]:
<tr>
<td></td>
<td style="text-align:right;">4.2 (26)<br/>
7.8 (50)<br/>
7.13 (55)<br/>
<span>9.14 (68)</span></td>
<td style="text-align:center;font-weight: bold;">Q1<br/>
Q2<br/>
Q3<br/>
Final</td>
<td>3.2 (20)<br/>
4.2 (26)<br/>
7.5 (47)<br/>
<span>10.8 (68)</span></td>
<td colspan="2" style="vertical-align:top;font-size:85%"><b>Umpires:</b> <a href="/wiki/Ray_Chamberlain_(umpire)" title="Ray Chamberlain (umpire)">Chamberlain</a>, <a href="/wiki/Brett_Rosebury" title="Brett Rosebury">Rosebury</a>, <a href="/wiki/Shaun_Ryan" title="Shaun Ryan">Ryan</a><br/>
<b><a href="/wiki/Norm_Smith_Medal" title="Norm Smith Medal">Norm Smith Medal</a>:</b> <a href="/wiki/Lenny_Hayes" title="Lenny Hayes">Lenny Hayes</a><br/>
<b>Television broadcast:</b> <a href="/wiki/Seven_Network" title="Seven Network">Seven</a> / <a href="/wiki/7mate" title="7mate">7mate</a><br/>
<b>National anthem:</b> Cameron and Taylor Henderson<br/></td>
</tr>
<tr>
<td>2 October (2:30 pm)</td>
<td style="font-weight:bold; text-align:right;"><a href="/wiki/Collingwood_Football_Club" title="Collingwood Football Club">Collingwood</a></td>
<td style="text-align:center;">def.</td>
<td><a href="/wiki/St_Kilda_Football_Club" title="St Kilda Football Club">St Kilda</a></td>
<td style="font-size: 100%"><a href="/wiki/Melbourne_Cricket_Ground" title="Melbourne Cricket Ground">MCG</a> (Crowd: 93,853)</td>
<td><a class="external text" href="http://www.afl.com.au/news/newsarticle/tabid/208/newsid/103599/default.aspx" rel="nofollow">Report</a></td>
</tr>
<tr>
<td></td>
<td style="text-align:right;">3.2 (20)<br/>
6.5 (41)<br/>
11.8 (74)<br/>
<span style="font-weight: bold;">16.12 (108)</span></td>
<td style="text-align:center;font-weight: bold;">Q1<br/>
Q2<br/>
Q3<br/>
Final</td>
<td>0.2 (2)<br/>
1.8 (14)<br/>
4.9 (33)<br/>
<span>7.10 (52)</span></td>
<td colspan="2" style="vertical-align:top;font-size:85%"><b>Umpires:</b> <a href="/wiki/Ray_Chamberlain_(umpire)" title="Ray Chamberlain (umpire)">Chamberlain</a>, <a href="/wiki/Brett_Rosebury" title="Brett Rosebury">Rosebury</a>, <a href="/wiki/Shaun_Ryan" title="Shaun Ryan">Ryan</a><br/>
<b><a href="/wiki/Norm_Smith_Medal" title="Norm Smith Medal">Norm Smith Medal</a>:</b> <a href="/wiki/Scott_Pendlebury" title="Scott Pendlebury">Scott Pendlebury</a><br/>
<b>Television broadcast:</b> <a href="/wiki/Seven_Network" title="Seven Network">Seven</a> / <a href="/wiki/7mate" title="7mate">7mate</a> / <a class="new" href="/w/index.php?title=Australia_Channel_(Internationally)&amp;action=edit&amp;redlink=1" title="Australia Channel (Internationally) (page does not exist)">Australia Channel (Internationally)</a><br/>
<b>National anthem:</b> <a href="/wiki/Julie_Anthony_(singer)" title="Julie Anthony (singer)">Julie Anthony</a><br/></td>
</tr>


In [ ]:
<tr>
<td>Saturday, 22 Sep (5:15 pm)</td>
<td align="right" style="font-weight: bold;"><a href="/wiki/Hawthorn_Football_Club" title="Hawthorn Football Club">Hawthorn</a> 13.19 (97)</td>
<td style="text-align:center;">def.</td>
<td><a href="/wiki/Adelaide_Football_Club" title="Adelaide Football Club">Adelaide</a> 14.8 (92)</td>
<td style="font-size: 85%"><a href="/wiki/Melbourne_Cricket_Ground" title="Melbourne Cricket Ground">MCG</a> (crowd: 69,146)</td>
<td style="font-size: 85%"><a class="external text" href="http://www.afl.com.au/match-centre/2012/26/haw-v-adel" rel="nofollow">Report</a></td>
</tr>

### FUNCTION TO SORT DATA INTO DICTIONARY FOR EACH MATCH

In [30]:
def match_function(match, this_round, this_year):

    try:

        namestocheck_good = ["Round", "Qualifying ", "Elimination", "Semi", "Preliminary", "Grand"]
        namestocheck_bad = ["NAB"]
        if any (name in this_round for name in namestocheck_good):
            #print ("entered inside" + this_round)

            ### WE ONLY WANT TO LOOK WHEN THE ROUND NAME INCLUDES THAT ABOVE
            test = match.find_all('td')

            # FUNCTION TAKE OFF FRONT PREFIX
            time_date= date_conversion(test[0].string)
            
            ### FUNCTION TO ASSIGN DAY IF IT EXISTS (BEFORE WE STRIP IT OFF FOR CLARITY AS NOT NEEDED)
            day = if_dayexists (time_date)
            
            ### FUNCTION TO TAKE OFF THE DAY AT THE START (AS NOT CONSISTENT)
            convert_time = day_conversion(time_date)
            
            #print ("this far 2")
            ### FUNCTION TO ENSURE CONSISTENT MONTH SYNTAX
            correct_time_syn = correct_month (convert_time)
            
            #print (correct_time_syn)
            ### FUNCTION TO ASSIGN DATE, MONTH AND TIME
            date = the_date(correct_time_syn)
            month = the_month(correct_time_syn)
            time = the_time(correct_time_syn)
            
            ### ADDS THE CURRENT YEAR
            convert_time_year = (str(this_year)+ " " + correct_time_syn)
            
            ### FUNCTION THAT CONVERTS TO TIME STRUCTURE
            struct_time = structured_times(convert_time_year)

            home_team = test[1].find('a').string
            away_team = test[3].find('a').string
            venue = test[4].find('a').string
            venue_crowd = (''.join(text for text in test[4].find_all(text=True) if text.parent.name != "a")).strip()
            venue_crowd_= (venue_crowd.split(':')[1])[:-1].strip()
            final_crowd = venue_crowd_.replace(",", "")
            match_report_url = test[5].find('a')['href']

            
            ### check if a score is return 
            test_for_score = (''.join(text for text in test[1].find_all(text=True) if text.parent.name != "a")).strip()

            if test_for_score:
                short_home_score = (''.join(text for text in test[1].find_all(text=True) if text.parent.name != "a")).strip()
                home_goals= short_home_score.split('.')[0]
                home_behinds= short_home_score.split('.')[1].split('(')[0]
                home_total = (short_home_score.split('.')[1].split('(')[1])[:-1]
                short_away_score = (''.join(text for text in test[3].find_all(text=True) if text.parent.name != "a")).strip()
                away_goals= short_away_score.split('.')[0]
                away_behinds= short_away_score.split('.')[1].split('(')[0]
                away_total = (short_away_score.split('.')[1].split('(')[1])[:-1]
                ### if we dont have any data for home goals (ie its a long list)
                return_dict = match_dictionary_return (day, date, month, this_year, this_round, time, struct_time, home_team, home_goals,home_behinds, home_total, away_team, away_goals, away_behinds, away_total, venue, final_crowd, match_report_url)

            
                return (return_dict)
                print ("yes")
            
            else: 
                print(match)
                ### if we have data for home goals (ie its a normal short list)
                ### add function to get the match score when its a big list
                ### pass in game[3]
                return_dict = match_dictionary_return (day, date, month, this_year, this_round, time, struct_time, home_team, "TBC",'TBC', 'TBC', "TBC", "TBC", "TBC", "TBC", venue, final_crowd, match_report_url)
                return (return_dict)

        
        else:
            print ("no name in scope", this_round)
    
    except Exception as e:
        print("this row does not have match data", e)
        #print ("/t",convert_time_year)
        #return False


### FUNCTION TO RETURN A LIST OF DATA FOR EACH ROUND

In [40]:
def round_function(round_number, year_data,current_year):    
    games= year_data[round_number].find_all('tr')
    #print (games)
    ## Find the round header
    round_head = (''.join(text for text in games[1].find_all(text=True) if text.parent.name != "a")).strip()
    #print (round_head)
    #if isinstance(round_head,str) == False:
    ## Check is string is empty- empty strings are falsy- so we are asking if not false- double negative- not false = true
    if not round_head:
        round_head = games[1].find('a').string

    this_round = []
    
 
    if round_head == "Grand Final" or round_head == "Grand Final Replay":
        #print (games[2])
        #print (games[3])
        ### we want to pass in games 3 aswell.....
        result= match_function(games[2],round_head, current_year)
        if result:
            this_round.append(result)
        return (this_round)

    else:  
        round_matchday = games[2:-1]
        for match in round_matchday:
            #print (match)
            result= match_function(match, round_head, current_year)
            if result:
                this_round.append(result)
        return (this_round)


### RETURN EACH ROUND FOR EACH YEAR FROM FUNCTIONS INTO MEGA LIST

In [41]:
mega_list = []

test_year = 2012

for year in list(range(test_year,test_year + 1)):
#for year in list(range(2009,2018,1)):
    current_year = str(year)
    url = find_url(current_year)
    req = requests.get(url)
    soup = BeautifulSoup(req.text, 'html.parser')
    ### FIND ALL OF THE TABLES THAT MATCH THE ROUND CRITERIA
    year_data = soup.find("div", {"id": "mw-content-text"}).find_all('table', {"style": "width: 100%; background-color: #f1f5fc; border: 2px solid #D0E5F5"})
    ### COUNT LENGTH OF ROUND TO ASSIGN TO LOOP
    round_range = len(year_data)
    
    for current_round in range(round_range):

        this_round = round_function(current_round, year_data, current_year)
        mega_list.extend(this_round)

print (mega_list)


this row does not have match data time data '2012 17 March 7:10\xa0pm' does not match format '%Y %d %B (%I:%M %p)'
this row does not have match data 'NoneType' object has no attribute 'count'
this row does not have match data 'NoneType' object has no attribute 'count'
this row does not have match data 'NoneType' object has no attribute 'count'
this row does not have match data 'NoneType' object has no attribute 'count'
this row does not have match data 'NoneType' object has no attribute 'count'
this row does not have match data 'NoneType' object has no attribute 'count'
this row does not have match data 'NoneType' object has no attribute 'count'
this row does not have match data 'NoneType' object has no attribute 'count'
this row does not have match data 'NoneType' object has no attribute 'count'
this row does not have match data 'NoneType' object has no attribute 'count'
<tr>
<td>Saturday, 29 September (2:30 pm)</td>
<td style="text-align:right;"><a href="/wiki/Hawthorn_Football_Club"

### TURN MEGALIST INTO DATAFRAME

In [414]:
df = pd.DataFrame(mega_list)
df.head(400)
#df.shape

,Away Behinds,Away Goals,Away Team,Away Total,Crowd,Date,Day,Home Behinds,Home Goals,Home Team,Home Total,Match Link,Month,Round,Structured Data,Time,Venue,Year
0,12,18,Carlton,120,72010,25,Thursday,10,9,Richmond,64,http://www.afl.com.au/news/newsarticle/tabid/2...,March,Round 1 (season launch),2010-03-25 19:10:00,7:10 pm,MCG,2010
1,16,13,Essendon,94,57772,26,Friday,11,19,Geelong,125,http://www.afl.com.au/news/newsarticle/tabid/2...,March,Round 1 (season launch),2010-03-26 19:40:00,7:40 pm,MCG,2010
2,15,17,Hawthorn,117,45615,27,Saturday,13,8,Melbourne,61,http://www.afl.com.au/news/newsarticle/tabid/2...,March,Round 1 (season launch),2010-03-27 14:10:00,2:10 pm,MCG,2010
3,10,12,West Coast,82,29201,27,Saturday,18,16,Brisbane Lions,114,http://www.afl.com.au/news/newsarticle/tabid/2...,March,Round 1 (season launch),2010-03-27 19:10:00,7:10 pm,Gabba,2010
4,6,15,St Kilda,96,31330,27,Saturday,10,13,Sydney,88,http://www.afl.com.au/news/newsarticle/tabid/2...,March,Round 1 (season launch),2010-03-27 19:10:00,7:10 pm,ANZ Stadium,2010
5,10,12,North Melbourne,82,21205,28,Sunday,18,13,Port Adelaide,96,http://www.afl.com.au/news/newsarticle/tabid/2...,March,Round 1 (season launch),2010-03-28 12:40:00,12:40 pm,AAMI Stadium,2010
6,8,9,Adelaide,62,30976,28,Sunday,16,17,Fremantle,118,http://www.afl.com.au/news/newsarticle/tabid/2...,March,Round 1 (season launch),2010-03-28 17:10:00,5:10 pm,Subiaco Oval,2010
7,15,19,Collingwood,129,49000,28,Sunday,15,13,Western Bulldogs,93,http://www.afl.com.au/news/newsarticle/tabid/2...,March,Round 1 (season launch),2010-03-28 14:10:00,2:10 pm,Etihad Stadium,2010
8,16,12,Carlton,88,36780,1,Thursday,11,16,Brisbane Lions,107,http://www.afl.com.au/news/newsarticle/tabid/2...,April,Round 2 (Easter),2010-04-01 19:40:00,7:40 pm,Gabba,2010
9,13,12,Melbourne,85,50421,3,Saturday,14,12,Collingwood,86,http://www.afl.com.au/news/newsarticle/tabid/2...,April,Round 2 (Easter),2010-04-03 14:10:00,2:10 pm,MCG,2010
